In [2]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.regularizers import l2
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split

In [3]:
# Load the Fashion MNIST dataset
(train_images, train_labels), (_, _) = tf.keras.datasets.fashion_mnist.load_data()

# Preprocess the images and labels
train_images = train_images.reshape(train_images.shape[0], 28, 28, 1).astype('float32') / 255.0
train_labels = np.ones(train_labels.shape)  # Label all Fashion MNIST images as real (1)

# Load the fake images and labels from the npy file
fake_images_file = 'Fake_Images.npy'
fake_images = np.load(fake_images_file)
fake_labels = np.zeros(len(fake_images))  # Label all fake images as fake (0)

In [9]:
# Combine the real and fake images and labels
combined_images = np.concatenate((train_images, fake_images), axis=0)
combined_labels = np.concatenate((train_labels, fake_labels), axis=0)

# Split the combined dataset into training and validation sets
train_images, val_images, train_labels, val_labels = train_test_split(
    combined_images, combined_labels, test_size=0.4)

In [ ]:
# Shuffle the combined dataset
combined_dataset = list(zip(combined_images, combined_labels))
np.random.shuffle(combined_dataset)
combined_images, combined_labels = zip(*combined_dataset)
combined_images = np.array(combined_images)
combined_labels = np.array(combined_labels)

In [ ]:
# Create data augmentation generator
datagen = ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.1,
    height_shift_range=0.1,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

In [ ]:
# Create the CNN model with dropout, L2 regularization, and batch normalization
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(28, 28, 1), kernel_regularizer=l2(0.01)),
    BatchNormalization(),
    Conv2D(32, (3, 3), activation='relu', kernel_regularizer=l2(0.01)),
    MaxPooling2D((2, 2)),
    Dropout(0.25),

    Conv2D(64, (3, 3), activation='relu', kernel_regularizer=l2(0.01)),
    BatchNormalization(),
    Conv2D(64, (3, 3), activation='relu', kernel_regularizer=l2(0.01)),
    MaxPooling2D((2, 2)),
    Dropout(0.25),

    Conv2D(128, (3, 3), activation='relu', kernel_regularizer=l2(0.01)),
    BatchNormalization(),
    MaxPooling2D((2, 2)),
    Dropout(0.25),

    Flatten(),
    Dense(256, activation='relu', kernel_regularizer=l2(0.01)),
    Dropout(0.5),
    Dense(1, activation='sigmoid')  # Binary output: 1 for real, 0 for fake
])

In [17]:
model.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_15 (Conv2D)          (None, 26, 26, 32)        320       
                                                                 
 batch_normalization_9 (Batc  (None, 26, 26, 32)       128       
 hNormalization)                                                 
                                                                 
 conv2d_16 (Conv2D)          (None, 24, 24, 32)        9248      
                                                                 
 max_pooling2d_9 (MaxPooling  (None, 12, 12, 32)       0         
 2D)                                                             
                                                                 
 dropout_12 (Dropout)        (None, 12, 12, 32)        0         
                                                                 
 conv2d_17 (Conv2D)          (None, 10, 10, 64)       

In [ ]:
# Compile the model
model.compile(optimizer=Adam(learning_rate=0.0002, beta_1=0.5), loss='binary_crossentropy', metrics=['accuracy'])

# Implement early stopping to prevent overfitting
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

In [ ]:
# Train the model using the training set with data augmentation and validate on the validation set
model.fit(datagen.flow(train_images, train_labels, batch_size=32),
          epochs=20,
          validation_data=(val_images, val_labels),
          callbacks=[early_stopping])

In [18]:
model.save('Trained_Cnn_Model_4.h5')